<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

## How can we analyze and compare shifts in demand and supply changes?

In this notebook, we will look for inflection points in the housing market in Tampa, Florida. We will analyze the supply and demand changes in the housing market.

#### What will you create in this notebook?

<p align="center">
  <img src="../../../images/tampa_yoy_supply_demand.png" alt="Alt text">
</p>

#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/experimental/supply_and_demand/yoy_supply_changes.ipynb)

### 1. Import the Parcl Labs Python Library

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs nbformat

In [1]:
import os
import pandas as pd
from parcllabs import ParclLabsClient
from parcllabs.beta.charting.styling import SIZE_CONFIG
from parcllabs.beta.charting.default_charts import create_dual_axis_chart

client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Search for Markets

In [2]:
# get tampa metro
tampa = client.search.markets.retrieve(
    query='Tampa',
    sort_by='TOTAL_POPULATION',
    sort_order='DESC',
    location_type='CITY',
    limit=1 # get 
)

tampa

,parcl_id,country,geoid,state_fips_code,name,state_abbreviation,region,location_type,total_population,median_income,parcl_exchange_market,pricefeed_market,case_shiller_10_market,case_shiller_20_market
0,5352995,USA,1271000,12,Tampa City,FL,SOUTH_ATLANTIC,CITY,388768,66802,0,1,0,0


In [3]:
# us parcl id
tampa_parcl_id = tampa['parcl_id'].tolist()

### 3. Retrieve the Data

In [4]:
# Lets get YoY changes for tampa market -- warning, you need a starter account to do this work. 
# this will pull back a lot of data and use a lot of credits. We are 
# capturing weekly supply numbers for 100 metros across the country. 

start_date='2022-09-01'

supply = client.for_sale_market_metrics.for_sale_inventory.retrieve(
    parcl_ids=tampa_parcl_id,
    start_date=start_date,
    limit=200
)

demand = client.market_metrics.housing_event_counts.retrieve(
    parcl_ids=tampa_parcl_id,
    start_date=start_date,
    limit=200
)

|████████████████████████████████████████| 1/1 [100%] in 0.2s (5.68/s) 
|████████████████████████████████████████| 1/1 [100%] in 0.2s (6.35/s) 


### 4. Prepare the data for analysis/charting

In [5]:
# supply is a weekly series while demand is a monthly series, need to truncate supply to the month
# and then compress it to a monthly series. We will take the median value of the month to represent
# that month
supply['date'] = supply['date'].dt.to_period('M').dt.to_timestamp()
supply = supply.groupby(['date', 'parcl_id'])['for_sale_inventory'].median().reset_index()

In [6]:
# now we can join with demand
data = pd.merge(demand[['date', 'parcl_id', 'sales']], supply[['date', 'parcl_id', 'for_sale_inventory']], on=['date', 'parcl_id'])
data

,date,parcl_id,sales,for_sale_inventory
0,2024-06-01,5352995,905,1837.5
1,2024-05-01,5352995,1128,1837.0
2,2024-04-01,5352995,1098,1683.0
3,2024-03-01,5352995,991,1613.5
4,2024-02-01,5352995,848,1442.5
5,2024-01-01,5352995,795,1250.0
6,2023-12-01,5352995,908,1423.5
7,2023-11-01,5352995,844,1542.5
8,2023-10-01,5352995,950,1519.0
9,2023-09-01,5352995,990,1443.0


In [7]:
data = data.sort_values('date')
data['pct_change_demand'] = data['sales'].pct_change(periods=12)
data['pct_change_supply'] = data['for_sale_inventory'].pct_change(periods=12)

### 5. Chart the Data

In [9]:
import pandas as pd
import plotly.graph_objects as go

from parcllabs.beta.charting.styling import default_style_config
from parcllabs.beta.charting.utils import create_labs_logo_dict, save_figure, sort_chart_data

def create_yoy_comparison_chart(
    title: str,
    line1_data: pd.DataFrame,
    line1_series: str,
    line2_data: pd.DataFrame,
    line2_series: str,
    save_path: str = None,
    yaxis_title: str = 'Year-over-Year % Change',
    height=675,
    width=1200,
    style_config=default_style_config
):
    fig = go.Figure()

    # Sort the data
    line1_data = sort_chart_data(line1_data)
    line2_data = sort_chart_data(line2_data)

    # Multiply the series values by 100 and round to 2 decimal points
    line1_data[line1_series] = (line1_data[line1_series] * 100).round(2)
    line2_data[line2_series] = (line2_data[line2_series] * 100).round(2)

    # Add the first line series
    fig.add_trace(go.Scatter(
        x=line1_data['date'],
        y=line1_data[line1_series],
        mode='lines+markers',
        line=dict(width=style_config['line_width'], color=style_config['line_color']),
        marker=dict(size=style_config['marker_size'], color=style_config['marker_color'], line=dict(width=1, color=style_config['marker_outline_color'])),
        name=line1_series
    ))

    # Add the second line series
    fig.add_trace(go.Scatter(
        x=line2_data['date'],
        y=line2_data[line2_series],
        mode='lines+markers',
        line=dict(width=style_config['line_width'], color=style_config['line2_color']),
        marker=dict(size=style_config['marker_size'], color=style_config['marker2_color'], line=dict(width=1, color=style_config['marker2_outline_color'])),
        name=line2_series
    ))

    fig.update_layout(
        margin=dict(l=40, r=40, t=80, b=40),
        height=height,
        width=width,
        title={
            'text': title,
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': style_config['title_font'],
        },
        plot_bgcolor=style_config['background_color'],
        paper_bgcolor=style_config['background_color'],
        font=dict(color=style_config['font_color']),
        xaxis=dict(
            title_text='',
            showgrid=style_config['showgrid'],
            gridwidth=style_config['gridwidth'],
            gridcolor=style_config['grid_color'],
            tickangle=style_config['tick_angle'],
            tickfont=style_config['axis_font'],
            linecolor=style_config['line_color_axis'],
            linewidth=style_config['linewidth'],
            titlefont=style_config['title_font_axis']
        ),
        yaxis=dict(
            title_text=yaxis_title,
            showgrid=style_config['showgrid'],
            gridwidth=style_config['gridwidth'],
            gridcolor=style_config['grid_color'],
            tickfont=style_config['axis_font'],
            tickprefix='',
            ticksuffix='%',
            zeroline=False,
            linecolor=style_config['line_color_axis'],
            linewidth=style_config['linewidth'],
            titlefont=style_config['title_font_axis']
        ),
        hovermode='x unified',
        hoverlabel=dict(
            bgcolor=style_config['hover_bg_color'],
            font_size=style_config['hover_font_size'],
            font_family=style_config['hover_font_family'],
            font_color=style_config['hover_font_color']
        ),
        legend=dict(
            x=style_config['legend_x'],
            y=style_config['legend_y'],
            xanchor=style_config['legend_xanchor'],
            yanchor=style_config['legend_yanchor'],
            font=style_config['legend_font'],
            bgcolor='rgba(0, 0, 0, 0)'
        ),
    )

    fig.add_layout_image(
        create_labs_logo_dict()
    )

    save_figure(fig, save_path=save_path, width=width, height=height)

    fig.show()

# Update style configuration for second line
default_style_config['line2_color'] = 'green'
default_style_config['marker2_color'] = 'green'
default_style_config['marker2_outline_color'] = 'green'
default_style_config['line_color'] = 'red'
default_style_config['marker_color'] = 'red'
default_style_config['marker_outline_color'] = 'red'

data = data.dropna()

data = data.rename(columns={'pct_change_demand': 'YoY Change in Demand', 'pct_change_supply': 'YoY Change in Supply'})

# Create the chart
create_yoy_comparison_chart(
    title='Tampa City: YoY Changes in Supply & Demand',
    line1_data=data,
    line1_series='YoY Change in Demand',
    line2_data=data,
    line2_series='YoY Change in Supply',
    save_path=None,
    style_config=default_style_config
)
